### Importando as bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

### Puxando o df

In [2]:
try:
    notebook_path = os.path.dirname(__file__) # Para scripts .py
except NameError:
    notebook_path = os.getcwd() # Para notebooks .ipynb

# Sobe na árvore de diretórios até encontrar a pasta raiz do projeto ('Challenge_ClickBus')
# Ele faz isso procurando por um arquivo/pasta que sempre existe na raiz, como o '.gitignore'
project_root = notebook_path
while not os.path.exists(os.path.join(project_root, '.gitignore')):
    project_root = os.path.dirname(project_root)
    if project_root == os.path.dirname(project_root): # Evita loop infinito
        raise FileNotFoundError("Não foi possível encontrar a raiz do projeto. Verifique se o arquivo '.gitignore' existe.")


# Constrói o caminho para a pasta de dados a partir da raiz
PROCESSED = os.path.join(project_root, 'data', 'processed') + '/'

print(f"Pasta de dados encontrada em: {PROCESSED}")

Pasta de dados encontrada em: c:\Users\pedro\Documents\Estudos\FIAP\Challenges\ClickBus\data\processed/


In [3]:
caminho = os.path.join(PROCESSED, 'transacoes_preparado.parquet')
df = pd.read_parquet(caminho)
df

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,Cliente 1,2018-12-26,15:33:35,Rodoviária 1,Rodoviária 1,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,Cliente 2,2018-12-05,15:07:57,Rodoviária 2,Rodoviária 2,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,155.97,1
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,Cliente 3,2018-12-21,18:41:54,Rodoviária 3,Rodoviária 3,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1
3,4dc44a6dd592b702feccb493d192210c86965aee684529...,Cliente 4,2018-12-06,14:01:38,Rodoviária 4,Rodoviária 4,0,0,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,1,55.22,1
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1
1741340,8bcfd696bb80f3433449c1442e4c6bab55d4bce9fe4311...,Cliente 126144,2019-08-01,10:06:37,Rodoviária 3,Rodoviária 406,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,106.93,2
1741341,1924fa1e8daf5821b3d062ac29d1ba56c70a54b41a177b...,Cliente 130048,2019-01-18,10:28:42,Rodoviária 1,Rodoviária 80,0,0,c17edaae86e4016a583e098582f6dbf3eccade8ef83747...,1,92.77,1
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1


### Pegando o range de data

In [4]:
data_max = df['date_purchase'].max()
data_min = df['date_purchase'].min()

print(f"Data range: {data_min} -> {data_max}")

Data range: 2013-09-12 00:00:00 -> 2024-04-01 00:00:00


### Qtd de vendas em cada ano

In [5]:
df['ano'] = df['date_purchase'].dt.year
df.groupby('ano')['fk_contact'].count()

ano
2013       341
2014     15182
2015     29651
2016     54618
2017     81354
2018    100277
2019    180935
2020    145669
2021    262649
2022    380073
2023    383163
2024    107425
Name: fk_contact, dtype: int64

### Receita gerada em cada ano

In [6]:
df.groupby('ano')['gmv_success'].sum().round(2)



ano
2013       42239.96
2014     2174842.43
2015     4479928.20
2016     8159603.11
2017    12746278.85
2018    15573402.84
2019    28893918.48
2020    21221801.83
2021    39339358.54
2022    63527848.61
2023    65351074.62
2024    19011906.55
Name: gmv_success, dtype: float64

In [7]:
anos_desejados = [2021, 2022, 2023, 2024]
df_filtrado = df[df['ano'].isin(anos_desejados)]

df_filtrado.shape

(1133310, 13)

### 2021 -> 2024 existe a maioria das vendas do df, então vamos pegar apenas essa maior parte do df

In [8]:
df = df[df['date_purchase'] >= '2021-01-01']
df

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021
5,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,Cliente 6,2021-02-11,22:25:15,Rodoviária 5,Rodoviária 6,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,154.20,1,2021
6,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,Cliente 7,2021-02-19,19:11:40,Rodoviária 6,Rodoviária 7,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021
9,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,Cliente 10,2021-07-02,11:41:19,Rodoviária 8,Rodoviária 10,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,61.55,1,2021
10,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,Cliente 11,2022-07-14,10:16:52,Rodoviária 9,Rodoviária 11,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,55.46,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741337,52a39652c7b3db7eedbba20c16b4e68df6f514bec1ad79...,Cliente 133072,2023-04-21,13:24:41,Rodoviária 1,Rodoviária 80,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,135.23,1,2023
1741338,655038f142aea0104f73671352b3304617b8d19801495a...,Cliente 131770,2023-04-08,10:49:02,Rodoviária 6,Rodoviária 18,0,0,f32828acecb4282c87eaa554d2e1db74e418cd68458430...,1,183.92,1,2023
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1,2023
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1,2023


### Volume financeiro no período

In [9]:
vlm_financeiro = df['gmv_success'].sum()
vlm_financeiro = round(vlm_financeiro, 2)
print(f"Volume financeiro: R${vlm_financeiro}")

Volume financeiro: R$187230188.32


### Criando coluna de qtd de compras de cada cliente

In [10]:
df['qtd_compras'] = df['fk_contact'].map(df['fk_contact'].value_counts())
df

C:\Users\pedro\AppData\Local\Temp\ipykernel_11972\2873861594.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['qtd_compras'] = df['fk_contact'].map(df['fk_contact'].value_counts())


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano,qtd_compras
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021,2
5,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,Cliente 6,2021-02-11,22:25:15,Rodoviária 5,Rodoviária 6,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,154.20,1,2021,3
6,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,Cliente 7,2021-02-19,19:11:40,Rodoviária 6,Rodoviária 7,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021,5
9,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,Cliente 10,2021-07-02,11:41:19,Rodoviária 8,Rodoviária 10,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,61.55,1,2021,8
10,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,Cliente 11,2022-07-14,10:16:52,Rodoviária 9,Rodoviária 11,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,55.46,1,2022,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741337,52a39652c7b3db7eedbba20c16b4e68df6f514bec1ad79...,Cliente 133072,2023-04-21,13:24:41,Rodoviária 1,Rodoviária 80,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,135.23,1,2023,2
1741338,655038f142aea0104f73671352b3304617b8d19801495a...,Cliente 131770,2023-04-08,10:49:02,Rodoviária 6,Rodoviária 18,0,0,f32828acecb4282c87eaa554d2e1db74e418cd68458430...,1,183.92,1,2023,14
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1,2023,59
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1,2023,59


In [11]:
df['total_gasto'] = df.groupby('fk_contact')['gmv_success'].transform('sum')
df['total_gasto'] = df['total_gasto'].round(2)
df

C:\Users\pedro\AppData\Local\Temp\ipykernel_11972\1675898690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_gasto'] = df.groupby('fk_contact')['gmv_success'].transform('sum')
C:\Users\pedro\AppData\Local\Temp\ipykernel_11972\1675898690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_gasto'] = df['total_gasto'].round(2)


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano,qtd_compras,total_gasto
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021,2,85.92
5,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,Cliente 6,2021-02-11,22:25:15,Rodoviária 5,Rodoviária 6,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,154.20,1,2021,3,576.64
6,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,Cliente 7,2021-02-19,19:11:40,Rodoviária 6,Rodoviária 7,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021,5,750.89
9,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,Cliente 10,2021-07-02,11:41:19,Rodoviária 8,Rodoviária 10,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,61.55,1,2021,8,1052.88
10,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,Cliente 11,2022-07-14,10:16:52,Rodoviária 9,Rodoviária 11,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,55.46,1,2022,16,787.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741337,52a39652c7b3db7eedbba20c16b4e68df6f514bec1ad79...,Cliente 133072,2023-04-21,13:24:41,Rodoviária 1,Rodoviária 80,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,135.23,1,2023,2,262.99
1741338,655038f142aea0104f73671352b3304617b8d19801495a...,Cliente 131770,2023-04-08,10:49:02,Rodoviária 6,Rodoviária 18,0,0,f32828acecb4282c87eaa554d2e1db74e418cd68458430...,1,183.92,1,2023,14,1562.74
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1,2023,59,10370.55
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1,2023,59,10370.55


### Criando o df de clientes

In [12]:
df_clientes = df[['fk_contact', 'total_gasto', 'qtd_compras']].drop_duplicates(subset='fk_contact')
df_clientes

,fk_contact,total_gasto,qtd_compras
4,Cliente 5,85.92,2
5,Cliente 6,576.64,3
6,Cliente 7,750.89,5
9,Cliente 10,1052.88,8
10,Cliente 11,787.83,16
...,...,...,...
1741290,Cliente 581808,569.57,1
1741303,Cliente 581809,76.78,1
1741315,Cliente 581811,82.48,1
1741329,Cliente 581812,119.82,1


### Criando df vendas

In [13]:
df_vendas = df
df_vendas = df.drop(columns=['total_gasto', 'qtd_compras'])
df_vendas

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021
5,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,Cliente 6,2021-02-11,22:25:15,Rodoviária 5,Rodoviária 6,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,154.20,1,2021
6,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,Cliente 7,2021-02-19,19:11:40,Rodoviária 6,Rodoviária 7,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021
9,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,Cliente 10,2021-07-02,11:41:19,Rodoviária 8,Rodoviária 10,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,61.55,1,2021
10,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,Cliente 11,2022-07-14,10:16:52,Rodoviária 9,Rodoviária 11,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,55.46,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741337,52a39652c7b3db7eedbba20c16b4e68df6f514bec1ad79...,Cliente 133072,2023-04-21,13:24:41,Rodoviária 1,Rodoviária 80,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,135.23,1,2023
1741338,655038f142aea0104f73671352b3304617b8d19801495a...,Cliente 131770,2023-04-08,10:49:02,Rodoviária 6,Rodoviária 18,0,0,f32828acecb4282c87eaa554d2e1db74e418cd68458430...,1,183.92,1,2023
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1,2023
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1,2023


### Separando em PF/PJ

In [14]:


df_vendas.query('total_tickets_quantity_success > 10')

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano
25089,d0e816e6fcf9701fb4efc41003a4ad32f0b11f70c78368...,Cliente 4879,2021-08-20,21:16:15,Rodoviária 144,Rodoviária 179,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,5f9c4ab08cac7457e9111a30e4664920607ea2c115a143...,733.82,12,2021
56637,71336e93caf97e6d690918ea8867a04aa0602400278929...,Cliente 29985,2023-02-01,16:49:17,Rodoviária 144,Rodoviária 179,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,609.28,12,2023
56641,fc4fec7a1e4593569cefabddbd9a763cf506efed18627e...,Cliente 29988,2023-01-19,00:34:45,Rodoviária 144,Rodoviária 179,0,0,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,516.01,12,2023
184735,a0a8b2c307858c775a5b54ccb80dc2e51152e791a87426...,Cliente 64281,2022-06-27,16:11:17,Rodoviária 144,Rodoviária 179,0,0,d6d824abba4afde81129c71dea75b8100e96338da5f416...,d6d824abba4afde81129c71dea75b8100e96338da5f416...,1150.83,12,2022
255505,a56472289d281e9a03c7d12deb8ec371bc4d296ff3f8b9...,Cliente 127231,2024-03-12,23:50:56,Rodoviária 144,Rodoviária 179,0,0,620c9c332101a5bae955c66ae72268fbcd397276617952...,e629fa6598d732768f7c726b4b621285f9c3b85303900a...,1404.11,12,2024
413396,687694b621505fd3927a9f5c523c2be548b408d14ff331...,Cliente 203337,2021-07-16,17:32:12,Rodoviária 144,Rodoviária 179,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1706.92,16,2021
453597,25e0329e03253761e0205c9e033386d2f2f2f8b98361de...,Cliente 214300,2022-06-12,00:31:14,Rodoviária 144,Rodoviária 179,0,0,e0f05da93a0f5a86a3be5fc0e301606513c9f7e59dac23...,620c9c332101a5bae955c66ae72268fbcd397276617952...,2829.61,12,2022
735481,f4a06a5a8dd65e4437cda779ab2af412af6f1a2410fdf9...,Cliente 328526,2022-12-23,07:52:26,Rodoviária 1,Rodoviária 481,Rodoviária 214,Rodoviária 3,d6d824abba4afde81129c71dea75b8100e96338da5f416...,d6d824abba4afde81129c71dea75b8100e96338da5f416...,1209.34,12,2022
740374,6819b71b513838ecfcf5a225a4fee695c789356df9c058...,Cliente 330450,2021-12-16,20:51:27,Rodoviária 372,Rodoviária 2150,Rodoviária 560,Rodoviária 398,cd70bea023f752a0564abb6ed08d42c1440f2e33e29914...,cd70bea023f752a0564abb6ed08d42c1440f2e33e29914...,320.50,12,2021
741170,03e535c6a74cc46a2e3fd403ae39c04be48bbdebeb3629...,Cliente 330786,2021-09-15,00:04:26,Rodoviária 1,Rodoviária 481,Rodoviária 214,Rodoviária 3,b4bbe448fde336bb6a7d7d765f36d3327c772b845e7b54...,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1083.76,16,2021


In [15]:
df_clientes['tp_pessoa'] = np.where(df_clientes['total_gasto'] >= 60000, 'PJ', 'PF')
df_clientes['tp_pessoa'].value_counts()

PF    375561
PJ        29
Name: tp_pessoa, dtype: int64

In [16]:
df_clientes

,fk_contact,total_gasto,qtd_compras,tp_pessoa
4,Cliente 5,85.92,2,PF
5,Cliente 6,576.64,3,PF
6,Cliente 7,750.89,5,PF
9,Cliente 10,1052.88,8,PF
10,Cliente 11,787.83,16,PF
...,...,...,...,...
1741290,Cliente 581808,569.57,1,PF
1741303,Cliente 581809,76.78,1,PF
1741315,Cliente 581811,82.48,1,PF
1741329,Cliente 581812,119.82,1,PF


### Colocando a data da ultima compra do cliente

In [17]:
df_clientes['ultima_compra'] = df_clientes['fk_contact'].map(df_vendas.groupby('fk_contact')['date_purchase'].max())
df_clientes

,fk_contact,total_gasto,qtd_compras,tp_pessoa,ultima_compra
4,Cliente 5,85.92,2,PF,2021-02-27
5,Cliente 6,576.64,3,PF,2021-07-02
6,Cliente 7,750.89,5,PF,2021-09-03
9,Cliente 10,1052.88,8,PF,2022-02-15
10,Cliente 11,787.83,16,PF,2022-07-14
...,...,...,...,...,...
1741290,Cliente 581808,569.57,1,PF,2022-02-26
1741303,Cliente 581809,76.78,1,PF,2023-10-12
1741315,Cliente 581811,82.48,1,PF,2022-12-18
1741329,Cliente 581812,119.82,1,PF,2022-02-10


### Fazendo o RFM de PF

In [18]:
df_rfm_pf = df_clientes[df_clientes['tp_pessoa'] == 'PF'].copy()
df_rfm_pf['recency'] = (df_rfm_pf['ultima_compra'].max() - df_rfm_pf['ultima_compra']).dt.days
df_rfm_pf['frequency'] = df_rfm_pf['qtd_compras']
df_rfm_pf['monetary'] = df_rfm_pf['total_gasto']
df_rfm_pf = df_rfm_pf.drop(columns=['tp_pessoa','ultima_compra', 'qtd_compras', 'total_gasto'])
df_rfm_pf

,fk_contact,recency,frequency,monetary
4,Cliente 5,1129,2,85.92
5,Cliente 6,1004,3,576.64
6,Cliente 7,941,5,750.89
9,Cliente 10,776,8,1052.88
10,Cliente 11,627,16,787.83
...,...,...,...,...
1741290,Cliente 581808,765,1,569.57
1741303,Cliente 581809,172,1,76.78
1741315,Cliente 581811,470,1,82.48
1741329,Cliente 581812,781,1,119.82


In [19]:
scaler = StandardScaler()
amostra = df_rfm_pf.sample(n=10000, random_state=42)
X_amostra = scaler.fit_transform(amostra[['recency', 'frequency', 'monetary']])

In [20]:
best_score = -1
best_k = 0

for k in range(2,11):
    modelo = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048)
    labels = modelo.fit_predict(X_amostra)
    score = silhouette_score(X_amostra, labels)
    print(f"Silhouette Score para k={k}: {score:.4f}")
    if score > best_score:
        best_score = score
        best_k = k
    
print(f"Melhor k: {best_k} com Silhouette Score: {best_score:.4f}")    


Silhouette Score para k=2: 0.4730
Silhouette Score para k=3: 0.4951
Silhouette Score para k=4: 0.5055
Silhouette Score para k=5: 0.4406
Silhouette Score para k=6: 0.4131
Silhouette Score para k=7: 0.3352
Silhouette Score para k=8: 0.3339


KeyboardInterrupt: 

In [ ]:
X_full = scaler.transform(df_rfm_pf[['recency', 'frequency', 'monetary']])
modelo_final = MiniBatchKMeans(n_clusters=best_k, random_state=42, batch_size=2048)
df_rfm_pf['cluster'] = modelo_final.fit_predict(X_full)
df_rfm_pf

,fk_contact,recency,frequency,monetary,cluster
4,Cliente 5,1129,2,85.92,0
5,Cliente 6,1004,3,576.64,0
6,Cliente 7,941,5,750.89,0
9,Cliente 10,776,8,1052.88,3
10,Cliente 11,627,16,787.83,3
...,...,...,...,...,...
1741290,Cliente 581808,765,1,569.57,3
1741303,Cliente 581809,172,1,76.78,2
1741315,Cliente 581811,470,1,82.48,3
1741329,Cliente 581812,781,1,119.82,0


In [ ]:
df_rfm_pf.groupby('cluster')[['recency', 'frequency', 'monetary']].mean()

,recency,frequency,monetary
cluster,,,
0,950.592456,1.469174,238.348405
1,159.476562,25.057292,3830.507425
2,142.977191,3.071695,507.256880
3,576.767439,1.929193,337.585866


In [ ]:
mapa_clusters = {
    0: 'Pessoa - Dormindo',
    1: 'Pessoa - Super Vip',
    2: 'Pessoa - Potencial',
    3: 'Pessoa - Quase dormindo'}

df_rfm_pf['cluster'] = df_rfm_pf['cluster'].map(mapa_clusters)
df_rfm_pf['cluster'].value_counts()
df_rfm_pf

,fk_contact,recency,frequency,monetary,cluster
4,Cliente 5,1129,2,85.92,Pessoa - Dormindo
5,Cliente 6,1004,3,576.64,Pessoa - Dormindo
6,Cliente 7,941,5,750.89,Pessoa - Dormindo
9,Cliente 10,776,8,1052.88,Pessoa - Quase dormindo
10,Cliente 11,627,16,787.83,Pessoa - Quase dormindo
...,...,...,...,...,...
1741290,Cliente 581808,765,1,569.57,Pessoa - Quase dormindo
1741303,Cliente 581809,172,1,76.78,Pessoa - Potencial
1741315,Cliente 581811,470,1,82.48,Pessoa - Quase dormindo
1741329,Cliente 581812,781,1,119.82,Pessoa - Dormindo


### Fazendo RFM de PJ

In [ ]:
df_clientes['grupo'] = df_clientes['fk_contact'].map(df_rfm_pf.set_index('fk_contact')['cluster'])
df_clientes['grupo'].value_counts()

Pessoa - Potencial         149898
Pessoa - Quase dormindo    124479
Pessoa - Dormindo           89664
Pessoa - Super Vip          11520
Name: grupo, dtype: int64

In [ ]:
df_rfm_pj = df_clientes[df_clientes['tp_pessoa'] == 'PJ'].copy()
df_rfm_pj['recency'] = (df_rfm_pj['ultima_compra'].max() - df_rfm_pj['ultima_compra']).dt.days
df_rfm_pj['frequency'] = df_rfm_pj['qtd_compras']
df_rfm_pj['monetary'] = df_rfm_pj['total_gasto']
df_rfm_pj = df_rfm_pj.drop(columns=['tp_pessoa','ultima_compra', 'qtd_compras', 'total_gasto', 'grupo'])
df_rfm_pj

,fk_contact,recency,frequency,monetary
2175,Cliente 1585,10,305,68015.59
2438,Cliente 1790,6,137,78207.51
25166,Cliente 11759,34,747,190209.93
25192,Cliente 11783,1,433,133072.71
25222,Cliente 11810,468,303,94523.59
38293,Cliente 19174,161,1203,308986.47
52531,Cliente 27214,306,142,103602.41
76680,Cliente 41355,213,389,129975.48
102958,Cliente 56456,606,100,61899.42
102971,Cliente 56467,515,372,107178.88


In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(df_rfm_pj[['recency', 'frequency', 'monetary']])

In [ ]:
best_k = 0
best_score = -1

for k in range(2,11):
    modelo = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048)
    labels = modelo.fit_predict(x)
    score = silhouette_score(x, labels)
    print(f"Silhouette Score para k={k}: {score:.4f}")
    if score > best_score:
        best_score = score
        best_k = k
        
print(f"Melhor k: {best_k} com Silhouette Score: {best_score:.4f}")

Silhouette Score para k=2: 0.4743
Silhouette Score para k=3: 0.5226
Silhouette Score para k=4: 0.5281
Silhouette Score para k=5: 0.4434
Silhouette Score para k=6: 0.4768
Silhouette Score para k=7: 0.4299
Silhouette Score para k=8: 0.3994
Silhouette Score para k=9: 0.3434
Silhouette Score para k=10: 0.3578
Melhor k: 4 com Silhouette Score: 0.5281


In [ ]:
df_rfm_pj['cluster'] = modelo_final.fit_predict(x)
df_rfm_pj.groupby('cluster')[['recency', 'frequency', 'monetary']].mean()

,recency,frequency,monetary
cluster,,,
0,126.000000,1089.333333,347971.346667
1,49.000000,210.846154,83867.200000
2,11.333333,659.666667,155158.380000
3,532.571429,338.000000,86741.501429


In [ ]:
mapa_niveis_pj = {
    0: "Empresa - VIP Consistente",
    1: "Empresa - Ativo Recente",
    2: "Empresa - Super Ativo",
    3: "Empresa - Dormindo"
}

df_rfm_pj['cluster'] = df_rfm_pj['cluster'].map(mapa_niveis_pj)

In [ ]:
df_rfm_pj['cluster'].value_counts()

Empresa - Ativo Recente      13
Empresa - Dormindo            7
Empresa - Super Ativo         6
Empresa - VIP Consistente     3
Name: cluster, dtype: int64

### Aplicando os GPs de PJ no df_cliente

In [ ]:
df_clientes.loc[df_clientes['tp_pessoa'] == 'PJ', 'grupo'] = (
    df_clientes.loc[df_clientes['tp_pessoa'] == 'PJ', 'fk_contact']
    .map(df_rfm_pj.set_index('fk_contact')['cluster'])
)
df_clientes['grupo'].value_counts()

Pessoa - Potencial           149898
Pessoa - Quase dormindo      124479
Pessoa - Dormindo             89664
Pessoa - Super Vip            11520
Empresa - Ativo Recente          13
Empresa - Dormindo                7
Empresa - Super Ativo             6
Empresa - VIP Consistente         3
Name: grupo, dtype: int64

### Descobrindo os endereços mais visitados

In [ ]:
df_vendas['place_destination_departure'].value_counts().head(10)

Rodoviária 18     137967
Rodoviária 17      82445
Rodoviária 33      42839
Rodoviária 9       38172
Rodoviária 16      30853
Rodoviária 42      27044
Rodoviária 94      23235
Rodoviária 91      15818
Rodoviária 160     13452
Rodoviária 69      12995
Name: place_destination_departure, dtype: int64

In [ ]:
df_cidades = df_vendas['place_origin_departure'].drop_duplicates()
df_cidades.shape

(2425,)

In [ ]:
df_cidades.duplicated().sum()

0

In [ ]:
df_vendas['place_origin_departure'] = df_vendas['place_origin_departure']
df_vendas['place_origin_departure'].value_counts().head(20)

Rodoviária 1      158867
Rodoviária 3       94468
Rodoviária 6       46163
Rodoviária 4       41660
Rodoviária 10      35867
Rodoviária 18      33754
Rodoviária 29      24268
Rodoviária 57      18384
Rodoviária 55      17745
Rodoviária 81      15843
Rodoviária 107     15296
Rodoviária 5       14241
Rodoviária 64      13259
Rodoviária 16      13016
Rodoviária 30      12712
Rodoviária 74      10353
Rodoviária 83      10142
Rodoviária 53       8985
Rodoviária 168      8509
Rodoviária 39       7772
Name: place_origin_departure, dtype: int64

### Gasto médio de cada cliente

In [ ]:
df_clientes['ticket_medio'] = df_clientes['total_gasto'] / df_clientes['qtd_compras']
df_clientes['ticket_medio'] = df_clientes['ticket_medio'].round(2)
df_clientes

,fk_contact,total_gasto,qtd_compras,tp_pessoa,ultima_compra,grupo,ticket_medio
4,Cliente 5,85.92,2,PF,2021-02-27,Pessoa - Dormindo,42.96
5,Cliente 6,576.64,3,PF,2021-07-02,Pessoa - Dormindo,192.21
6,Cliente 7,750.89,5,PF,2021-09-03,Pessoa - Dormindo,150.18
9,Cliente 10,1052.88,8,PF,2022-02-15,Pessoa - Quase dormindo,131.61
10,Cliente 11,787.83,16,PF,2022-07-14,Pessoa - Quase dormindo,49.24
...,...,...,...,...,...,...,...
1741290,Cliente 581808,569.57,1,PF,2022-02-26,Pessoa - Quase dormindo,569.57
1741303,Cliente 581809,76.78,1,PF,2023-10-12,Pessoa - Potencial,76.78
1741315,Cliente 581811,82.48,1,PF,2022-12-18,Pessoa - Quase dormindo,82.48
1741329,Cliente 581812,119.82,1,PF,2022-02-10,Pessoa - Dormindo,119.82


### Exportando DFs para CSV

In [ ]:
CSV = os.getenv('CSV_PATH')
df_clientes.to_csv(f'{CSV}cliente.csv', index=False)

In [ ]:
df_vendas.to_csv(f'{CSV}vendas.csv', index=False)

### Guardando Cluster em uma Variavel

In [ ]:
# Unifica os dataframes de pessoa física e jurídica
df_clusters_completo = pd.concat([df_rfm_pf, df_rfm_pj], axis=0)

# --- ALTERAÇÃO APLICADA AQUI ---
# Agrupa por cluster e calcula a média, mínimo e máximo para cada métrica RFM
df_clusters_agregado = df_clusters_completo.groupby('cluster').agg(
    recency_mean=('recency', 'mean'),
    recency_min=('recency', 'min'),
    recency_max=('recency', 'max'),
    frequency_mean=('frequency', 'mean'),
    frequency_min=('frequency', 'min'),
    frequency_max=('frequency', 'max'),
    monetary_mean=('monetary', 'mean'),
    monetary_min=('monetary', 'min'),
    monetary_max=('monetary', 'max')
).reset_index()

# Junta a contagem de clientes em cada cluster
qtd_clientes = df_clientes['grupo'].value_counts().reset_index()
qtd_clientes.columns = ['cluster', 'Qtd']
df_clusters_final = pd.merge(df_clusters_agregado, qtd_clientes, on='cluster')


# Salva o resultado final nos caminhos desejados
CAMINHO_WEBHOOK = os.path.join(project_root,'Desafios', 'data', 'webhook') + '/'
df_clusters_final.to_csv(f'{CAMINHO_WEBHOOK}cluster.csv', index=False)
df_clusters_final.to_csv(f'{CSV}cluster.csv', index=False)

print("Arquivo de clusters gerado com sucesso, incluindo métricas de Média, Mínimo e Máximo.")
display(df_clusters_final)

Arquivo de clusters gerado com sucesso, incluindo métricas de Média, Mínimo e Máximo.


,cluster,recency_mean,recency_min,recency_max,frequency_mean,frequency_min,frequency_max,monetary_mean,monetary_min,monetary_max,Qtd
0,Empresa - Ativo Recente,49.000000,0,213,210.846154,69,389,83867.200000,63374.33,129975.48,13
1,Empresa - Dormindo,532.571429,306,746,338.000000,100,714,86741.501429,61899.42,107178.88,7
2,Empresa - Super Ativo,11.333333,1,34,659.666667,433,933,155158.380000,133072.71,193058.02,6
3,Empresa - VIP Consistente,126.000000,0,217,1089.333333,1022,1203,347971.346667,280698.56,454229.01,3
4,Pessoa - Dormindo,950.592456,775,1186,1.469174,1,21,238.348405,4.19,4400.94,89664
5,Pessoa - Potencial,142.977191,0,502,3.071695,1,22,507.256880,5.00,3809.98,149898
6,Pessoa - Quase dormindo,576.767439,359,874,1.929193,1,22,337.585866,5.00,3946.68,124479
7,Pessoa - Super Vip,159.476562,0,1180,25.057292,1,437,3830.507425,333.66,58158.68,11520


### Visualizando as empresas_onibus

In [ ]:
df_vendas

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success,ano
4,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,Cliente 5,2021-02-23,20:08:25,Rodoviária 3,Rodoviária 5,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,45.31,1,2021
5,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,Cliente 6,2021-02-11,22:25:15,Rodoviária 5,Rodoviária 6,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,154.20,1,2021
6,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,Cliente 7,2021-02-19,19:11:40,Rodoviária 6,Rodoviária 7,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,188.99,1,2021
9,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,Cliente 10,2021-07-02,11:41:19,Rodoviária 8,Rodoviária 10,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,61.55,1,2021
10,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,Cliente 11,2022-07-14,10:16:52,Rodoviária 9,Rodoviária 11,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,55.46,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741337,52a39652c7b3db7eedbba20c16b4e68df6f514bec1ad79...,Cliente 133072,2023-04-21,13:24:41,Rodoviária 1,Rodoviária 80,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,135.23,1,2023
1741338,655038f142aea0104f73671352b3304617b8d19801495a...,Cliente 131770,2023-04-08,10:49:02,Rodoviária 6,Rodoviária 18,0,0,f32828acecb4282c87eaa554d2e1db74e418cd68458430...,1,183.92,1,2023
1741339,4e4f5fcded3e8b6915e05c6b9b504ec6830f723a831f68...,Cliente 133024,2023-12-29,19:47:12,Rodoviária 6,Rodoviária 18,0,0,3068430da9e4b7a674184035643d9e19af3dc7483e31cc...,1,156.00,1,2023
1741342,18b75368642c6c347fd00f56cd98322bb6d920b6a07545...,Cliente 133024,2023-01-08,23:57:55,Rodoviária 6,Rodoviária 18,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,294.99,1,2023
